In [2]:
os.chdir('..')
os.listdir()

[
    'app',
    'LICENSE',
    'requirements.txt',
    'Dockerfile',
    'pyproject.toml',
    'README.md',
    '.gitignore',
    'docker-compose.yml',
    '.ipynb_checkpoints',
    'poetry.lock',
    '.git',
    '.vscode',
    'data',
    'heroku.yml',
    'notebooks'
]

In [17]:
import csv
import os
import requests
import pandas as pd  # only through miniforge conda env
from rich import pretty
from rich.progress import track
from geopy.geocoders import ArcGIS
from geopy import distance

pretty.install()

In [8]:
df = pd.read_csv('data/CookCountyMEArchives.csv')
print(df.columns)
df.head()

Index(['Age', 'Casenumber', 'Cold Related', 'Commissioner District',
       'Death Date', 'Gender', 'Gunrelated', 'Heat Related', 'Incident City',
       'Incident Date', 'Incident Street', 'Incident Zip', 'Latino',
       'Latitude', 'Longitude', 'Manner', 'Opioids', 'Primarycause',
       'Primarycause Linea', 'Race', 'Residence City', 'Residence Zip',
       'Secondarycause'],
      dtype='object')
/Users/nick/miniforge3/envs/geo/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (21) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Age,Casenumber,Cold Related,Commissioner District,Death Date,Gender,Gunrelated,Heat Related,Incident City,Incident Date,...,Latitude,Longitude,Manner,Opioids,Primarycause,Primarycause Linea,Race,Residence City,Residence Zip,Secondarycause
0,38.0,ME2021-03896,False,5.0,2021-04-13T03:02:00.000,Male,NaN,False,PARK FOREST,2021-04-01T15:00:00.000,...,41.502759,-87.687533,NaN,NaN,NaN,NaN,Black,Park Forest,60466,NaN
1,67.0,ME2021-03895,False,NaN,2021-04-13T02:59:00.000,Female,NaN,False,DES PLAINES,2021-04-13T02:40:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,Asian,Des Plaines,60016,NaN
2,70.0,ME2021-03894,False,16.0,2021-04-13T02:02:00.000,Male,NaN,False,BERWYN,2021-02-18T15:00:00.000,...,41.843009,-87.796797,NaN,NaN,NaN,NaN,White,Berwyn,60402,NaN
3,35.0,ME2021-03893,False,6.0,2021-04-13T00:30:00.000,Female,NaN,False,LYNWOOD,2021-04-13T00:16:00.000,...,41.534453,-87.557231,NaN,NaN,NaN,NaN,Black,Lynwood,60411,NaN
4,78.0,ME2021-03892,False,NaN,2021-04-12T23:10:00.000,Male,NaN,False,HARVEY,2021-04-06T23:57:00.000,...,NaN,NaN,NaN,NaN,NaN,NaN,Black,Harvey,60426,NaN


In [25]:
def geo_query(row):
    street = row['Incident Street']
    city = row['Incident City']
    zip_code = row['Incident Zip']
    address = f"{street} {city} {zip_code}"
    print(address)
    url = f"https://my-geocoder.herokuapp.com/geocode?address={requests.utils.quote(address)}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise ValueError("Invalid response")


def self_geocode(address: str) -> tuple[float, float]:

    address = f"{street} {city} {zip_code}"
    coded = geocoder.geocode(address)
    return {
        "latitude": coded.latitude,
        "longitude": coded.longitude
    }

def make_address(row) -> str:
    street = row['Incident Street']
    city = row['Incident City']
    zip_code = row['Incident Zip']
    return f"{street} {city} {zip_code}"



In [27]:
df['address'] = df.apply(lambda row: make_address(row), axis=1)
df.address.values[:5]

array(['321 SPRINGFIELD ST. PARK FOREST 60466',
       '772 SANDBORN STREET DES PLAINES 60016',
       '6924 26TH ST. BERWYN 60402', '19901 PARK AVE, 1 N LYNWOOD 60411',
       '15600 HONORE AVENUE HARVEY 60426'], dtype=object)

In [33]:
from geopy.extra.rate_limiter import RateLimiter
from tqdm import tqdm
tqdm.pandas()

geolocator = ArcGIS()
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=0)
df['geo_location'] = df['address'].progress_apply(geocode)


100%|██████████| 52710/52710 [5:34:56<00:00,  2.62it/s]


In [39]:
df['new_lat'] = df['geo_location'].apply(lambda x: x.latitude if x else None)
df['new_long'] = df['geo_location'].apply(lambda x: x.longitude if x else None)


In [45]:
def calc_distance(row):
    if not pd.isna(row.new_lat) and not pd.isna(row.new_long) and not pd.isna(row.Latitude) and not pd.isna(row.Longitude):
        p1 = (row.new_lat, row.new_long)
        p2 = (row.Latitude, row.Longitude)
        d = distance.distance(p1, p2)
        return d.miles
    else:
        return None


df['distance'] = df.apply(lambda row: calc_distance(row), axis=1)

In [50]:
df.distance.values[:5]


array([3.16807908e-05,            nan, 3.92842776e-05, 2.69696344e-05,
                  nan])

In [61]:
df.to_csv('data/coded_records.csv', index=False)

In [57]:
import pickle

with open('data/dataframe.pkl', 'wb') as f:
    pickle.dump(df, f)